In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

2024-04-11 19:26:39.818996: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 19:26:39.819132: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 19:26:39.947810: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
dataset = "/kaggle/input/brain-tumor-classification-mri"

images = []
labels = []

def preprocess_image(img):
    img = img/255
    img = np.float32(img)
    img = cv2.resize(img, (224, 224))
    img = img - np.mean(img)
    img = img/np.std(img)
    return img

for data_dir_name in os.listdir(dataset):
    # training or testing directory
    data_dir = os.path.join(dataset, data_dir_name)
    for label in os.listdir(data_dir):
        label_dir = os.path.join(data_dir, label)
        for img_name in os.listdir(label_dir):
            img_dir = os.path.join(label_dir, img_name)
            img = cv2.imread(img_dir)
            img = preprocess_image(img)
            images.append(img)
            labels.append(label)
            
images = np.asarray(images)
labels = np.asarray(labels)

In [ ]:
print(f"images shape -> {images.shape}")
print(f"labels shape -> {labels.shape}")

In [ ]:
def augment_images(images, labels, augmentation_obj):
    images_after_aug = []
    labels_after_aug = []
    for index, image in enumerate(images):
        for i in range(3):
            img = next(augmentation_obj.flow(np.reshape(image, (1, 224, 224, 3))))
            images_after_aug.append(np.reshape(img, (224, 224, 3)))
            labels_after_aug.append(labels[index])
    return np.asarray(images_after_aug), np.asarray(labels_after_aug)

augmentation_obj = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=0.2,
                                                                   zoom_range=0.01,
                                                                   horizontal_flip=True,
                                                                   vertical_flip=True,
                                                                   rescale=None)
images, labels = augment_images(images, labels, augmentation_obj)

In [ ]:
print(f"images shape -> {images.shape}")
print(f"labels shape -> {labels.shape}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=101)

In [ ]:
label_map = {"no_tumor": 0, "glioma_tumor": 1, "meningioma_tumor": 2, "pituitary_tumor": 3}

def one_hot_encode(y, label_map):
    y_new = []
    for label in y:
        y_new.append(label_map[label])
    y_new = tf.keras.utils.to_categorical(y_new)
    return y_new

y_train = one_hot_encode(y_train, label_map)
y_test = one_hot_encode(y_test, label_map)

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
base_model = DenseNet121(include_top= False,
                         weights= "imagenet",
                         input_shape= (224, 224, 3))
model = tf.keras.layers.Dropout(0.8)(base_model.output)
model = tf.keras.layers.GlobalAveragePooling2D(name= "Extraction_layer")(model)
model = tf.keras.layers.Dropout(0.8)(model)
model = tf.keras.layers.Dense(4, 
                              activation= 'softmax', 
                              kernel_regularizer= tf.keras.regularizers.L2(0.0001),
                              bias_regularizer= tf.keras.regularizers.L2(0.0001))(model)
model = tf.keras.models.Model(inputs= base_model.input, outputs= model)

In [ ]:
model.compile(optimizer= tf.keras.optimizers.Adam(0.001), 
              loss= "categorical_crossentropy", 
              metrics= ["accuracy"])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, 
                    y_train, 
                    epochs= 32, 
                    batch_size= 16, 
                    validation_data= (X_test, y_test), 
                    callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor= 'val_loss', factor= 0.1, mode= 'min', patience= 2),
                                 tf.keras.callbacks.EarlyStopping(patience= 14, monitor= 'val_loss', mode= 'min', restore_best_weights=True)]
                   )